# **SMART WELLNESS ADVISOR**

# **📌 1. Install Dependencies**

In [ ]:
!pip install flask-ngrok pandas numpy scikit-learn joblib


# **📌 2. Generate Synthetic Dataset**

In [ ]:
import numpy as np, pandas as pd, random
random.seed(42); np.random.seed(42)

def generate_row():
    age = int(np.clip(np.random.normal(30,8),16,80))
    bmi = round(np.random.normal(25,4),1)
    sleep = round(np.random.normal(7,1.5),1)
    activity = np.random.choice(['low','moderate','high'], p=[0.4,0.4,0.2])
    water = round(np.random.normal(1.8,0.5),1)
    sugar = np.random.choice(['low','high'], p=[0.7,0.3])
    stress = np.random.choice(['low','medium','high'], p=[0.5,0.35,0.15])
    score=0
    if bmi>27: score+=2
    elif bmi>24: score+=1
    if activity=='low': score+=2
    elif activity=='moderate': score+=1
    if sleep<6: score+=1
    if sugar=='high': score+=1
    if stress=='high': score+=1
    risk = 'high' if score>=4 else ('moderate' if score>=2 else 'low')
    return [age,bmi,sleep,activity,water,sugar,stress,risk]

cols=["age","bmi","sleep_hours","activity_level","water_intake_l","sugar_intake","stress_level","risk"]
df = pd.DataFrame([generate_row() for _ in range(1200)], columns=cols)
df.to_csv("lifestyle_dataset.csv", index=False)
df.head()


,age,bmi,sleep_hours,activity_level,water_intake_l,sugar_intake,stress_level,risk
0,33,24.4,8.0,low,2.6,low,low,moderate
1,32,29.0,6.1,low,1.5,low,medium,high
2,16,22.8,5.5,low,2.0,low,low,moderate
3,24,27.4,10.8,low,2.0,low,low,high
4,25,23.8,6.1,low,2.7,low,low,moderate


# **📌 3. Train Machine Learning Model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib, pandas as pd

df = pd.read_csv("lifestyle_dataset.csv")
enc = {}
df2 = df.copy()
cat_cols = ["activity_level","sugar_intake","stress_level"]
for c in cat_cols:
    le = LabelEncoder(); df2[c] = le.fit_transform(df2[c]); enc[c]=le
le_y = LabelEncoder(); df2["risk"] = le_y.fit_transform(df2["risk"]); enc["risk"]=le_y

X = df2.drop(columns=["risk"]); y = df2["risk"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
model = RandomForestClassifier(n_estimators=150, max_depth=8, random_state=42)
model.fit(X_train,y_train)
preds = model.predict(X_test)
print("Acc:", accuracy_score(y_test,preds))
print(classification_report(y_test, preds, target_names=enc["risk"].classes_))
joblib.dump(model, "rf_model.joblib")
joblib.dump(enc, "encoders.joblib")
print("Saved rf_model.joblib and encoders.joblib")


Acc: 0.9541666666666667
              precision    recall  f1-score   support

        high       1.00      0.91      0.95        74
         low       0.97      0.93      0.95        40
    moderate       0.93      0.99      0.96       126

    accuracy                           0.95       240
   macro avg       0.97      0.94      0.95       240
weighted avg       0.96      0.95      0.95       240

Saved rf_model.joblib and encoders.joblib


# **📌 STEP 4 — Create Hybrid KRR + ML Engine**

In [ ]:
import joblib, numpy as np
model = joblib.load("rf_model.joblib")
enc = joblib.load("encoders.joblib")

def ml_predict(user):
    X = np.array([
        float(user.get("age",30)),
        float(user.get("bmi",25.0)),
        float(user.get("sleep_hours",7.0)),
        float(user.get("water_intake_l",1.8)),
        int(enc["activity_level"].transform([user["activity_level"]])[0]),
        int(enc["sugar_intake"].transform([user["sugar_intake"]])[0]),
        int(enc["stress_level"].transform([user["stress_level"]])[0])
    ]).reshape(1,-1)
    y_pred = model.predict(X)[0]
    label = enc["risk"].inverse_transform([y_pred])[0]
    probs = model.predict_proba(X)[0]
    prob_dict = {enc["risk"].classes_[i]: float(probs[i]) for i in range(len(probs))}
    return label, prob_dict

def apply_rules(user, ml_label):
    label = ml_label; recs=[]
    try:
        bmi = float(user.get("bmi",0))
        sleep = float(user.get("sleep_hours",7))
        water = float(user.get("water_intake_l",1.8))
        activity = user.get("activity_level","moderate")
        sugar = user.get("sugar_intake","low")
        stress = user.get("stress_level","low")
    except:
        return ml_label, []

    if bmi>=30 and activity=="low":
        label="high"; recs.append("High BMI & low activity — consult provider and increase activity.")
    if sleep<5 and label!="high":
        label = "moderate" if label=="low" else label
        recs.append("Improve sleep: aim for 7-8 hours.")
    if sugar=="high" and label=="moderate":
        label="high"; recs.append("Reduce sugar intake.")
    if stress=="high":
        recs.append("Practice stress reduction (exercise, mindfulness).")
    if water<1.2:
        recs.append("Increase daily water intake to 1.5–2 L.")
    # ensure unique
    recs = list(dict.fromkeys(recs))
    return label, recs

def assess(user):
    ml_label, probs = ml_predict(user)
    final_label, recs = apply_rules(user, ml_label)
    return {"ml_label": ml_label, "ml_probs": probs, "final_label": final_label, "recommendations": recs}


# **📌 STEP 5 — Launch Public Flask API Using ngrok**

In [ ]:
from flask import Flask, request, render_template_string, jsonify
app = Flask(__name__)

HTML = """
<!doctype html><html><head><meta charset="utf-8"><title>Smart Wellness Advisor</title>
<style>body{font-family:Arial;padding:20px} label{display:block;margin:8px 0} input,select{padding:6px;width:240px}
.btn{margin-top:12px;padding:8px 12px}</style></head><body>
<h2>Smart Wellness Advisor (Demo)</h2>
<form method="post" action="/assess">
  <label>Age: <input name="age" type="number" value="30" required></label>
  <label>BMI: <input name="bmi" step="0.1" value="25.0" required></label>
  <label>Sleep hours: <input name="sleep_hours" step="0.1" value="7.0" required></label>
  <label>Activity level:
    <select name="activity_level">
      <option value="low">low</option><option value="moderate" selected>moderate</option><option value="high">high</option>
    </select>
  </label>
  <label>Water intake (L): <input name="water_intake_l" step="0.1" value="1.8" required></label>
  <label>Sugar intake:
    <select name="sugar_intake"><option value="low" selected>low</option><option value="high">high</option></select>
  </label>
  <label>Stress level:
    <select name="stress_level"><option value="low" selected>low</option><option value="medium">medium</option><option value="high">high</option></select>
  </label>
  <button class="btn" type="submit">Assess</button>
</form>
{% if result %}
  <hr>
  <h3>Result</h3>
  <p><strong>ML prediction:</strong> {{ result['ml_label'] }} (probs: {{ result['ml_probs'] }})</p>
  <p><strong>Final label after rules:</strong> {{ result['final_label'] }}</p>
  <p><strong>Recommendations:</strong></p>
  <ul>{% for r in result['recommendations'] %}<li>{{ r }}</li>{% endfor %}</ul>
{% endif %}
</body></html>
"""

@app.route('/', methods=['GET'])
def home():
    return render_template_string(HTML)

@app.route('/assess', methods=['POST'])
def assess_form():
    data = {k: request.form.get(k) for k in ['age','bmi','sleep_hours','activity_level','water_intake_l','sugar_intake','stress_level']}
    # convert numeric fields
    data['age']=int(data['age']); data['bmi']=float(data['bmi']); data['sleep_hours']=float(data['sleep_hours'])
    data['water_intake_l']=float(data['water_intake_l'])
    result = assess(data)
    return render_template_string(HTML, result=result)

@app.route('/api/assess', methods=['POST'])
def api_assess():
    payload = request.get_json(force=True)
    # validate/complete defaults
    for k in ['age','bmi','sleep_hours','water_intake_l','activity_level','sugar_intake','stress_level']:
        if k not in payload:
            payload[k] = {"age":30,"bmi":25.0,"sleep_hours":7.0,"water_intake_l":1.8}.get(k, "low")
    res = assess(payload)
    return jsonify(res)


# **CELL 6 — Start server thread (run once), then run the iframe cell**

In [ ]:
# Start Flask in a background thread. Run this cell ONCE.
import threading, time
def start_server():
    app.run(port=5000, host="0.0.0.0")
threading.Thread(target=start_server, daemon=True).start()
time.sleep(2)
print("Flask started on port 5000")


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Flask started on port 5000


In [ ]:
from google.colab import output
# Use iframe preview (or serve_kernel_port_as_window to open new tab)
output.serve_kernel_port_as_iframe(5000, '/')


<IPython.core.display.Javascript object>